#Views and CTEs on Databricks

In this demonstration, you will creante and explore views and common table expressions (CTEs)

##Learning Objectives
By the end of this lesson, you should be able to:
- Use Spark SQL DDL to define views
- Run queries that use common table expressions

We start by creating a table of data we can use for the demonstration

In [0]:
%sql
CREATE TABLE external_table
USING CSV OPTIONS(
  path '/Volumes/workspace/default/csv_files/us_harvard_ned_presidential_elections.csv',
  header 'true',
  mode 'FAILFAST'
);

SELECT * FROM external_table;

In [0]:
%python
display(dbutils.fs.ls("dbfs:/FileStore/shared_uploads/correia.alefe@gmail.com/"))

In [0]:
# 1. Definir o caminho do Volume que você criou
caminho_volume = "/Volumes/workspace/default/csv_files/us_harvard_ned_presidential_elections.csv"

# 2. Ler o arquivo (o Spark aceita caminhos de Volume nativamente aqui)
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(caminho_volume)

# 3. Salvar como uma tabela Delta real no Unity Catalog
# Isso vai criar a tabela dentro do seu catálogo e esquema 'default'
df.write.mode("overwrite").saveAsTable("workspace.default.elections_delta")

print("Sucesso! A tabela 'elections_delta' foi criada e está pronta para uso.")

In [0]:
%sql
SELECT * FROM elections_delta LIMIT 10;

To show a list of tables(and views), we use the **SHOW TABLES** command also demonstrated below.

In [0]:
%sql
SHOW TABLES;

##Views, Temp Views & Global Temp Views##

To set this demonstration up, we are going to first create one of eacht type of view.

Then in the next notebook, we weill explore the differences between how each one behaves.

###Views
Let's create a view that contains only the data where the origin is "ABQ" and the destination is "LAX".

In [0]:
%sql
CREATE VIEW bra_presidents AS
  SELECT * FROM elections_delta
  WHERE country_id = 'BRA'
  ORDER BY year;

SELECT * FROM bra_presidents;

Note that the view has been added to the list below

In [0]:
%sql
SHOW TABLES;

###Temporary Views

Next we'll create a temporary view.

The syntax is very similar but adds **TEMPORARY** to the command.

In [0]:
%sql
CREATE TEMPORARY VIEW bra_recent AS
  SELECT * FROM bra_presidents
  WHERE year > 2000
  ORDER BY year;

SELECT * FROM bra_recent;


Now if we show our tables again, we sill see the one table and both views/

Make note of the value in the **isTemporary** column.

In [0]:
%sql
SHOW TABLES;

###Global Temp Views

Lastly, we'll create a global temp view.

Here we simply add **GLOBAL** to the command.

Also note the **global_temp** database qualifier in the subsequent **SELECT** statement.

In [0]:
%sql
CREATE GLOBAL TEMPORARY VIEW pt_brazil AS
  SELECT * FROM bra_presidents WHERE party_1 = 'Partido dos Trabalhadores (Workers’ Party)';

SELECT * FROM global_temp.pt_brazil;